## EXPLORATION PRELIMINAIRE DU CORPUS

### PARCOURS DES DOCUMENTS 

#### Exploration des documents: répartition selon les journaux, nombres d'exemplaires, dates...

In [ ]:
#Imports 
from collections import defaultdict
import os

import matplotlib.pyplot as plt
import numpy as np

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

##### Vérification du nombre de document dans le corpus

In [ ]:
path = "../data/caf/"
files = sorted(os.listdir(path))
len(files)
#Nous avons un fichier supplémentaire "README.txt" ce qui justifie le nombre total

##### Manipulation du nom des fichiers pour réléver les informations de publication ( date et source)

In [ ]:
all_years = [str(year) for year in range(1835, 1969)]

In [ ]:
count_decade = defaultdict(int)
count_month = defaultdict(int)
count_newspapers = defaultdict(int)
covered_years = set()

for f in files:
    if "_" in f and f.endswith("txt"):
        elems = f.split("_")
        
        newspaper = elems[1]
        
        year = elems[2].split("-")[0]
        covered_years.add(year)
        decade = year[:3] + "0s"
        
        month = int(elems[2].split("-")[1])
        
        count_decade[decade] += 1
        count_newspapers[newspaper] += 1
        count_month[month] += 1
        
    else:
        print(f"Anomalous file: {f}")

In [ ]:
# Classement des articles selon les jouneaux qui les ont publiés 

print(f"Il y a {count_newspapers['JB421']} exemplaires du journal L'Avenir du Luxembourg, {count_newspapers['JB427']} exemplaires de La Libre Belgique, {count_newspapers['JB555']} exemplaires de L'Indépendance belge, {count_newspapers['JB555A']} exemplaires de L'Indépendance belge (édité en Angleterre), {count_newspapers['JB567']} exemplaires du Journal de Bruxelles, {count_newspapers['JB572']} exemplaires du Journal de Charleroi, {count_newspapers['JB638']} exemplaires de La Meuse, {count_newspapers['JB685']} exemplaires du journal Le petit Bleu, {count_newspapers['JB2729']} exemplaires du journal Le Vingtième Siècle, {count_newspapers['JB773']} exemplaire de Vers l'Avenir, {count_newspapers['JB837']} exemplaires du journal Le Peuple, {count_newspapers['JB838']} exemplaire du journal Le Soir et {count_newspapers['JB1051']} exemplaires du journal Le Drapeau Rouge")

In [ ]:
# liste des années sans aucun article sur le sujet 

missing_years = [y for y in all_years if y not in covered_years]
print(f"Années manquantes: {', '.join(missing_years)}")

#### Analyse de la distribution du vocabulaire du corpus 

##### Création de notre liste de stopwords

In [ ]:
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout", 
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "autre", "jusqu", "déjà", "heures", "très", "puis", 
       "vers", "fit", "trop", "dès", "chambre", "maison", "hui", "dem", "louer", "etc", 
       "peu", "leurs", "ans", "ceux", "rue", "ecr", "voici", "celui", "cctto", "qu'il", "d'un",
       "jour", "francs", "bon", "quelques", "soir", "toute", "saint", "mois", "chez"]
sw = set(sw)

In [ ]:
print(f"{len(sw)} stopwords:\n {sorted(sw)}")

##### regroupement de nos fichiers en un nouveau fichier .txt

In [ ]:
!cat ../data/caf/*.txt > ../data/allcaf.txt

##### Tokenisation du nouveau fichier

In [ ]:
# Récupération du contenu du fichier
path = "../data/allcaf.txt"
limit = 10**8

with open(path, 'r', encoding='utf-8') as f:
    text = f.read()[:limit]

In [ ]:
# Tokenization
words = nltk.wordpunct_tokenize(text)
print(f"{len(words)} words found")

In [ ]:
# On affiche les 10 premiers mots 
words[:10]

##### Représentation du vocabulaire 

In [ ]:
# Elimination les stopwords et les termes non alphabétiques
kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
voc = set(kept)
print(f"{len(kept)} words kept ({len(voc)} different word forms)")

Récupération des mots les plus fréquents pour en faire un plot

In [ ]:
#Mots les plus fréquent
fdist = nltk.FreqDist(kept)
fdist.most_common(10)

In [ ]:
# Plot: les n mots les plus fréquents
n = 20
fdist.plot(n, cumulative=True)

### NETTOYAGE DU CORPUS

#### Filtrage du corpus

##### Imports

In [ ]:
#imports 
import sys
import re
from nltk.tokenize import sent_tokenize

##### Répertoire d'inputs et d'outputs

In [ ]:
# Le répertoire qui contient vos fichiers txt exportés de Camille
indir = "../data"
# Le répertoire qui contiendra les fichiers txt nettoyés
outdir = "../data/clean"

if not os.path.exists(outdir):
    os.mkdir(outdir)

##### Termes de recherches dans le corpus 

In [ ]:
query = ["artiste"]

##### Extraction de phrases contenant le terme de la recherche

In [ ]:
# Création d'une regex afin de trouver les mots de la liste query dans le texte
regex = re.compile(f"\\b({'|'.join(query)})\\b", re.IGNORECASE)

In [ ]:
for file in os.listdir(indir)[:10]:
    if file.endswith(".txt"):
        relevant_sentences = []
        f_in = open(os.path.join(indir, file), encoding="utf-8")
        text = f_in.read()
        for sentence in sent_tokenize(text):
            if regex.search(sentence):
                relevant_sentences.append(sentence)
        f_in.close()
        f_out = open(os.path.join(outdir, file), "w", encoding="utf-8")
        f_out.write("\n\n".join(relevant_sentences))
        f_out.close()

#### Fonction de Nettoyage

##### Paramètres

In [ ]:
# Stockage du contenu du fichier dans une liste
content_list = []
path = "../data/clean/1900.txt"
limit = 10**8
with open(path, 'r', encoding='utf-8') as f:
        content_list.append(f.read())

In [ ]:
file = content_list

In [ ]:
# Impression des 200 premiers caractères du contenu du premier fichier
content_list[0:200]

In [ ]:
# Ecriture de tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(path, 'w', encoding='utf-8') as f:
    f.write(' '.join(content_list))

In [ ]:
# Impression du contenu du fichier et constat des "déchets"
with open(path, 'r', encoding='utf-8') as f:
    before = f.read()

before[:500]

##### Application de la fonction de nettoyage

In [ ]:
year = 1900

In [ ]:
def clean_text(year, folder=None):
    if folder is None:
        input_path = f"{year}.txt"
        output_path = f"{year}_clean.txt"
    else:
        input_path = f"{folder}/{year}.txt"
        output_path = f"{folder}/{year}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.upper() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

##### Fonction sur le fichier complet 

In [ ]:
 
clean_text(year, folder=temp_path)

In [ ]:
# Vérification du résultat
with open(os.path.join(temp_path, f'{year}_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

after[:500]

## ANALYSE APPROFONDIE ET ENRICHISSEMENT

### Extraction de mots clés 

##### Imports

In [ ]:
import yake

##### Extraction 

In [ ]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=20)
kw_extractor

In [ ]:
# Lister les Fichiers
data_path = "../data/clean"
files = [f for f in os.listdir(data_path) if f.endswith('.txt')]

In [ ]:
this_file = files[0]
this_file

In [ ]:
# Récupérer le texte du fichier
text = open(os.path.join(data_path, this_file), 'r', encoding='utf-8').read()
text[:500]

In [ ]:
# Extraire les mots clés de ce texte
keywords = kw_extractor.extract_keywords(text)
keywords

In [ ]:
# Ne garder que les bigrammes
kept = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 2:
        kept.append(kw)
kept

### Création du nuage de mots 

##### imports

In [ ]:
from collections import Counter
from wordcloud import WordCloud
from IPython.display import Image

##### Nuage de mots

In [ ]:
frequencies = Counter(after.split())
print(frequencies.most_common(10))

In [ ]:
cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"{year}.png"))
Image(filename=os.path.join(temp_path, f"{year}.png"))

### Reconnaissance et extraction d'entités nommées 

##### imports

In [ ]:
from collections import defaultdict
import sys
import spacy
from spacy.lang.fr.examples import sentences

In [ ]:
nlp = spacy.load('fr_core_news_md')

In [ ]:
# Charger le texte
n=100000
text = open("../data/clean/1900.txt", encoding='utf-8').read()[:n]

In [ ]:
%%time
# Traiter le texte

doc = nlp(text)

#### Entités personnes 

In [ ]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1

In [ ]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=False)

for person, freq in sorted_people[:100]:
    print(f"{person} apparait {freq} fois dans le corpus")

#### Entités Lieux

In [ ]:
# Compter les entités
location = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "LOC" and len(ent.text) > 3:
        location[ent.text] += 1

In [ ]:
# Trier et imprimer

sorted_location = sorted(location.items(), key=lambda kv: kv[1], reverse=True)

for location, freq in sorted_location[:50]:
    print(f"{location} apparait {freq} fois dans le corpus")

#### Entités organisations

In [ ]:
# Compter les entités
organisation = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "ORG" and len(ent.text) > 3:
        organisation[ent.text] += 1

In [ ]:
# Trier et imprimer

sorted_organisation = sorted(organisation.items(), key=lambda kv: kv[1], reverse=True)

for organisation, freq in sorted_organisation[:50]:
    print(f"{organisation} apparait {freq} fois dans le corpus")

### Analyse de sentiments

In [ ]:
#imports 
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

#### fonction get_sentiment

In [ ]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

def get_sentiment(input_text):
    blob = tb(input_text)
    polarity, subjectivity = blob.sentiment
    polarity_perc = f"{100*abs(polarity):.0f}"
    subjectivity_perc = f"{100*subjectivity:.0f}"
    if polarity > 0:
        polarity_str = f"{polarity_perc}% positive"
    elif polarity < 0:
        polarity_str = f"{polarity_perc}% negative"
    else:
        polarity_str = "neutral"
    if subjectivity > 0:
        subjectivity_str = f"{subjectivity}% subjective"
    else:
        subjectivity_str = "perfectly objective"
    print(f"This text is {polarity_str} and {subjectivity_str}.")

#### Analyse du sentiment de quelques phrases 

In [ ]:
get_sentiment("Ida Rubinstein jooie le rôle, moina en femme douloureuse qu'en artiste ayant un souci constant de l'esthétique.")

In [ ]:
get_sentiment("Podlesnaia, qui incarne la femme fatale est très belle, et comme femme, et comme artiste.")

In [ ]:
get_sentiment("Tout le monde, en France, aime Claudette, parce que c'est une grande, une véritable artiste, doublée d'une femme simple, avenante, gentille.")

In [ ]:
get_sentiment("Régina Bruylant est une jeune artiste qui, après un an et demi /e travail, vient d'ouvrir une exposition dans une modeste salle de la chaussée de Ninove.")

In [ ]:
get_sentiment("Madeleine Deltenre n'en reste pas moins une véritable artiste vibrante et sincère dont la personnalité ne tardera pas, nous en sommes persuadés, à s'imposer.")

In [ ]:
get_sentiment("Françoise Rosay est une femme intelligente, une artiste de talent et l'excellente maman de trois grands garçons.")

In [ ]:
get_sentiment("Mlle dc a Bruchollerie mille fois mieux qu'une virtuose : simplement et grandement une artiste >.")

In [ ]:
get_sentiment("Mme Madeleine Soria prête au personnage de Zaza sa personnalité de parfaite artiste et de femme sensible, jouant son rôle de manière profondément humaine")

In [ ]:
get_sentiment("La Christiani (le nom d'une célèbre artiste, une femme qui joue du violoncelle comme Servais, Demunck ou Batta),")

In [ ]:
get_sentiment("On ne nous donnera plus une Hélène belle commo WlDelvil, était une artiste délibérée et une femme jolie, sach mt draper do distinction dos airs de luronne.")

In [ ]:
get_sentiment("Mme de JRudder, — encore une femme artiste, d'un enthousiasme et d'une vaillance incomparables et à laquelle nous consacrerons un iour une étude complète,")

#### Utilisation des transformers

In [ ]:
#librairies et imports 
!pip install tensorflow
!pip install sentencepiece
!pip install transformers

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline

#### Chargement du modèle 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine", use_pt=True)
model = TFAutoModelForSequenceClassification.from_pretrained("tblard/tf-allocine")

sentiment_analyser = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [ ]:
sentiment_analyser("Bonne, généreuse .charitable, d'une' piété éclairée, d'une grande, intelligence, amijé, des arts, artiste elle-même, c'était une: femme vrajpnent supérieuse.")

In [ ]:
sentiment_analyser("Mademoiselle VERA GORSKA Artiste dramatique La jeune femme justifiait ce litre par les rôles de figurante qu'elle remplissait quelquefois dans les théâtres de troisième ordre.")

In [ ]:
sentiment_analyser("Je suis tout d'abord, et avant tout, une artiste, je ne suis femme que secondairement.")

In [ ]:
sentiment_analyser("L'aetrice.quelque admirable qu'elle soit comme artiste, reste femme")

In [ ]:
sentiment_analyser("Elle- est artiste et mutine; elle est femme et cependant poupée ; elle est gracieuse ; elle est belle.")

In [ ]:
sentiment_analyser("Une femme, une faible femme, artiste elle-même, a vaincu l'entêtement du P. Boom.")

In [ ]:
sentiment_analyser("Contre elle, il y a sa jeunesse : une qualité chez la femme, un défaut pour l'artiste !")